In [1]:
import numpy as np
import logging
from gensim.models import Word2Vec
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

from nergrit_non_contextual_embeddings import NergritNonContextualEmbeddings


app = NergritNonContextualEmbeddings()

In [2]:
app.init_data(ignore_case=True)

saving result to cache '.cache/nergrit:train_df.pkl'
saving result to cache '.cache/nergrit:validation_df.pkl'
saving result to cache '.cache/nergrit:test_series.pkl'
saving result to cache '.cache/nergrit:test_sentence_series.pkl'
saving result to cache '.cache/nergrit:uncased_vocab_series.pkl'


In [3]:
w2vmodel = app.build_word2vec_model()

saving result to cache '.cache/nergrit-non-contextual-embeddings:word2vec_model.pkl'


In [4]:
X_train, y_train = app.build_Xy_train(w2vmodel)
X_validation, y_validation = app.build_Xy_validation(w2vmodel)

saving result to cache '.cache/nergrit-non-contextual-embeddings:train_tags.pkl'
saving result to cache '.cache/nergrit-non-contextual-embeddings:word2vec_train.pkl'
saving result to cache '.cache/nergrit-non-contextual-embeddings:validation_tags.pkl'
saving result to cache '.cache/nergrit-non-contextual-embeddings:word2vec_validation.pkl'


In [5]:
print(np.shape(X_train), np.shape(y_train))
print(np.shape(X_validation), np.shape(y_validation))

(56210, 100) (56210,)
(6983, 100) (6983,)


In [6]:
clf = GaussianNB()

In [7]:
clf.fit(X_train, y_train)

GaussianNB()

In [8]:
y_pred = clf.predict(X_validation)

In [9]:
le = app.get_tags_label_encoder()
pred_labels = le.inverse_transform(y_pred)

In [10]:
accuracy = accuracy_score(y_validation, y_pred)
precision = precision_score(y_validation, y_pred, average='macro')
recall = recall_score(y_validation, y_pred, average='macro')
f1 = f1_score(y_validation, y_pred, average='macro')

print("Accuracy = {0:.2f}".format(accuracy))
print("Precision = {0:.2f}".format(precision))
print("Recall = {0:.2f}".format(recall))
print("F1 = {0:.2f}".format(f1))

Accuracy = 0.80
Precision = 0.42
Recall = 0.21
F1 = 0.23
